# Final GB Polyp

In [1]:
import keras
import os
import sys
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from keras import models
from keras import layers
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16, VGG19, InceptionV3, InceptionResNetV2
from keras.applications import InceptionResNetV2
from keras.layers import Dense, GlobalAveragePooling2D, Input, Dropout
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from imgaug import augmenters as iaa
from random import shuffle

%matplotlib inline
#plt.rcParams['figure.figsize'] = (10, 10)#(30, 20) 

target_width = 299
target_height = 299
target_size = (target_width, target_height)
batch_size = 256
transfer_learning_epochs = 1000
fine_tuning_epochs = 200
train_set_size = 4137
validation_set_size = 1033
test_set_size = 885
dropout_rate = 0.5
learning_rate = 0.0001
decay = 0.0001

# jarvis
#data_dir = '/sdb1/share/gbpolyp/final_data'
#work_dir = '/home/jarvis/workspace/gbpolyp'
#profundus
data_dir = '/share/gbpolyp/final_data'
work_dir = '/home/obsk/workspace/gbpolyp'
# abdomen
#data_dir = '/home/ubuntu/workspace/share/gbpolyp/final_data'
#work_dir = '/home/ubuntu/workspace/gbpolyp'

train_dir = os.path.join(data_dir, 'train')
validation_dir = os.path.join(data_dir, 'validation')
test_dir = os.path.join(data_dir, 'test')
csv_file = os.path.join(data_dir, 'clinical_data.csv')

pretrained_wt_dir = os.path.join(work_dir, 'pretrained_wt')
log_dir = os.path.join(work_dir, 'log')

image_checkpoint_path = os.path.join(pretrained_wt_dir, '1591-0.8519.hdf5')
clinical_data_checkpoint_path = os.path.join(pretrained_wt_dir, 'clinical_data_weights.hdf5')
ensemble_checkpoint_path = os.path.join(pretrained_wt_dir, '618-0.8858.hdf5')

# Augmentation parameters
seq = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontally flip 50% of the images
    iaa.Flipud(0.5),
    iaa.Crop(percent=(0, 0.1)),
    iaa.Multiply((0.9, 1.1)),
    iaa.Affine(
        translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, #{"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, 
        rotate=(-40, 40),
        shear=(-20, 20),
        order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
        cval=0,
        mode='edge'#['constant', 'edge']
    ),
    iaa.OneOf([
        iaa.GaussianBlur((0, 3.0)),
        iaa.AverageBlur(k=(2, 7)),
        iaa.Sharpen(alpha=(0, 0.1), lightness=(0.75, 1.5))
    ]),
])

/home/obsk/anaconda3/envs/naesa1.8/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Helper functions

In [2]:
def plot(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

    
def plot_smoothly(history):
    def smooth_curve(points, factor=0.8):
      smoothed_points = []
      for point in points:
        if smoothed_points:
          previous = smoothed_points[-1]
          smoothed_points.append(previous * factor + point * (1 - factor))
        else:
          smoothed_points.append(point)
      return smoothed_points

    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(len(acc))
    
    plt.plot(epochs,
             smooth_curve(acc), 'bo', label='Smoothed training acc')
    plt.plot(epochs,
             smooth_curve(val_acc), 'b', label='Smoothed validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.figure()

    plt.plot(epochs,
             smooth_curve(loss), 'bo', label='Smoothed training loss')
    plt.plot(epochs,
             smooth_curve(val_loss), 'b', label='Smoothed validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()
    
    
# define roc_callback, inspired by https://github.com/keras-team/keras/issues/6050#issuecomment-329996505
def auc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.contrib.metrics.streaming_auc(y_pred, y_true)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value

def plot_learning_rate(epochs, learning_rate, decay):
    epoch_list = [i+1 for i in range(epochs)]
    lr_list = []
    for epoch in range(epochs):
        learning_rate = learning_rate * 1/(1 + decay * epoch)
        lr_list.append(learning_rate)

    plt.plot(epoch_list, lr_list)

# Custom Generator

In [3]:
def preprocess_input(x, is_train=False):
    x /= 255.
#    x -= 0.5
#    x *= 2.
    
    if is_train: x = seq.augment_images(x)
        
    return x

def get_input(file_path):
    img = image.load_img(file_path, target_size=target_size)
    x = image.img_to_array(img)
    
    return x


def get_output(path):
    class_name = os.path.basename(os.path.dirname(path))
    if class_name == 'neoplastic':
        labels = 1
    else:
        labels = 0
    return labels


def get_data(file_path, df):
    # img
    img = image.img_to_array(image.load_img(file_path, target_size=target_size))

    # num
    name = os.path.basename(file_path)[:-9]
    try:
        size = df[df['name']==name].iloc[0]['size']
        multiplicity = df[df['name']==name].iloc[0]['multiplicity']
        age = df[df['name']==name].iloc[0]['age']
    except IndexError:
        print(name)
        size = 0
        multiplicity = 0
        age = 0
    num = (size, multiplicity, age)
    
    # label
    class_name = os.path.basename(os.path.dirname(file_path))
    if class_name == 'neoplastic':
        label = 1
    else:
        label = 0
    
    return img, num, label

    
def custom_generator(file_list, data_frame, batch_size=batch_size, target_size=target_size, is_train=False, non_image_data=False):
    if is_train: shuffle(file_list)
    
    i = 0
    
    while True:
        # Select files (paths/indices) for the batch
        if i+batch_size < len(file_list):
            batch_paths = file_list[i:i+batch_size]
        else:
            batch_paths = file_list[i:]
            
        img_list = []
        num_list = []
        label_list = [] 
          
        # Read in each input, perform preprocessing and get labels
        for file_path in batch_paths:
            (img, num, label) = get_data(file_path, df)
            #img = get_input(input_path)
            #label = get_output(input_path)
            img_list.append(img)
            num_list.append(num)
            label_list.append(label)
            
        # Return a tuple of (input,output) to feed the network
        batch_img = preprocess_input(np.array(img_list), is_train)
        batch_num = np.array(num_list)
        batch_label = np.array(label_list)
        
        if i+batch_size >= len(file_list):
            i = 0
        else:
            i = i+batch_size
        
        if non_image_data:
            yield ([batch_img, batch_num], batch_label)
        else:
            yield (batch_img, batch_label)

# Data providers

In [4]:
def get_clinical_data(csv_file):
    df = pd.read_csv(csv_file)
    df['name'] = df['name'].apply(lambda x: x.split('\\')[-1].split('.')[0])

    d1 = df[df['test']==0]

    size_mean = d1['size'].mean()
    size_std = d1['size'].std()
    df['size'] = df['size'].apply(lambda x: (x - size_mean) / size_std)

    age_mean = d1['age'].mean()
    age_std = d1['age'].std()
    df['age'] = df['age'].apply(lambda x: (x - age_mean) / age_std)
    
    return df


def get_image_data(dir_name, df):
    count = 0
    for path, dir, files in os.walk(dir_name):
            count += len(files)
    print('total images = %d' % count)

    file_list = []
    img_data = np.ndarray((count, target_width, target_height, 3), dtype=np.float32)
    clinical_data = np.ndarray((count, 3), dtype=np.float32)
    y_data = np.ndarray((count, ), dtype=np.int8)

    i = 0

    for path, dir, files in os.walk(dir_name):
        for file_name in files:
            file_path = os.path.join(path, file_name)
            img = image.load_img(file_path, target_size=(target_width, target_height))
            x = image.img_to_array(img)
            img_data[i] = x
            if os.path.basename(path) == 'neoplastic': y_data[i] = 1
            else: y_data[i] = 0
            name = os.path.basename(file_path)[:-9]
            try:
                size = df[df['name']==name].iloc[0]['size']
                multiplicity = df[df['name']==name].iloc[0]['multiplicity']
                age = df[df['name']==name].iloc[0]['age']
            except IndexError:
                print(name)
                size = 0
                multiplicity = 0
                age = 0
            clinical_data[i] = (size, multiplicity, age)
            file_list.append(file_path)
            i += 1

    return file_list, img_data, clinical_data, y_data

# Load Image Pre-trained Weights

In [5]:
conv_base = InceptionV3(weights='imagenet',
                  include_top=False,
                  input_shape=(target_width, target_height, 3))

i1 = conv_base.input
x = conv_base(i1)
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=i1, outputs=predictions)

for layer in conv_base.layers:
    layer.trainable = False

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=learning_rate),
              metrics=['acc'])

model.load_weights(image_checkpoint_path)

layer_dict = dict([(layer.name, layer) for layer in model.layers])
for layer in model.layers:
    print(layer.name, layer.trainable_weights)

input_1 []
inception_v3 []
global_average_pooling2d_1 []
dense_1 [<tf.Variable 'dense_1/kernel:0' shape=(2048, 1024) dtype=float32_ref>, <tf.Variable 'dense_1/bias:0' shape=(1024,) dtype=float32_ref>]
dense_2 [<tf.Variable 'dense_2/kernel:0' shape=(1024, 1) dtype=float32_ref>, <tf.Variable 'dense_2/bias:0' shape=(1,) dtype=float32_ref>]


In [6]:
image_pretained_weights = layer_dict['dense_1'].get_weights() # shape=(2048, 1024)

# Load Clinical Data Pre-trained Weights

In [7]:
model = models.Sequential()
model.add(layers.Dense(4, activation='relu', input_dim=3))
model.add(layers.Dense(4, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer=RMSprop(lr=0.0001), loss='binary_crossentropy', metrics=['acc'])

model.load_weights(clinical_data_checkpoint_path)

layer_dict = dict([(layer.name, layer) for layer in model.layers])
for layer in model.layers:
    print(layer.name, layer.trainable_weights)

dense_3 [<tf.Variable 'dense_3/kernel:0' shape=(3, 4) dtype=float32_ref>, <tf.Variable 'dense_3/bias:0' shape=(4,) dtype=float32_ref>]
dense_4 [<tf.Variable 'dense_4/kernel:0' shape=(4, 4) dtype=float32_ref>, <tf.Variable 'dense_4/bias:0' shape=(4,) dtype=float32_ref>]
dense_5 [<tf.Variable 'dense_5/kernel:0' shape=(4, 1) dtype=float32_ref>, <tf.Variable 'dense_5/bias:0' shape=(1,) dtype=float32_ref>]


In [8]:
clinical_data_weights1 = layer_dict['dense_3'].get_weights() # shape=(3, 4)
clinical_data_weights2 = layer_dict['dense_4'].get_weights() # shape=(4, 4)

# Define Ensemble Model

In [5]:
conv_base = InceptionV3(weights='imagenet',
                  include_top=False,
                  input_shape=(target_width, target_height, 3))

i1 = conv_base.input
x1 = conv_base(i1)
x1 = GlobalAveragePooling2D()(x1)
x1 = Dense(1024, activation='relu')(x1)

i2 = Input(shape=(3,))
x2 = Dense(4, activation='relu',)(i2)
x2 = Dense(4, activation='relu')(x2)


x = layers.concatenate([x1, x2])
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=([i1, i2]), outputs=predictions)
for layer in conv_base.layers:
    layer.trainable = False
    
layer_dict = dict([(layer.name, layer) for layer in model.layers])
for layer in model.layers:
    print(layer.name, layer.trainable_weights)

input_1 []
inception_v3 []
input_2 []
global_average_pooling2d_1 []
dense_2 [<tf.Variable 'dense_2/kernel:0' shape=(3, 4) dtype=float32_ref>, <tf.Variable 'dense_2/bias:0' shape=(4,) dtype=float32_ref>]
dense_1 [<tf.Variable 'dense_1/kernel:0' shape=(2048, 1024) dtype=float32_ref>, <tf.Variable 'dense_1/bias:0' shape=(1024,) dtype=float32_ref>]
dense_3 [<tf.Variable 'dense_3/kernel:0' shape=(4, 4) dtype=float32_ref>, <tf.Variable 'dense_3/bias:0' shape=(4,) dtype=float32_ref>]
concatenate_3 []
dense_4 [<tf.Variable 'dense_4/kernel:0' shape=(1028, 1) dtype=float32_ref>, <tf.Variable 'dense_4/bias:0' shape=(1,) dtype=float32_ref>]


In [6]:
model.load_weights(ensemble_checkpoint_path)

In [10]:
layer_dict['dense_6'].set_weights(image_pretained_weights) # shape=(2048, 1024)
layer_dict['dense_7'].set_weights(clinical_data_weights1) # shape=(3, 4)
layer_dict['dense_8'].set_weights(clinical_data_weights2) # shape=(4, 4)

layer_dict['dense_6'].trainable = False # shape=(2048, 1024)
layer_dict['dense_7'].trainable = False # shape=(3, 4)
layer_dict['dense_8'].trainable = False # shape=(4, 4)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
inception_v3 (Model)            (None, 8, 8, 2048)   21802784    input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 3)            0                                            
__________________________________________________________________________________________________
global_average_pooling2d_2 (Glo (None, 2048)         0           inception_v3[1][0]               
__________________________________________________________________________________________________
dense_7 (D

# Train Ensemble Model

In [7]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
inception_v3 (Model)            (None, 8, 8, 2048)   21802784    input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 3)            0                                            
__________________________________________________________________________________________________
global_average_pooling2d_1 (Glo (None, 2048)         0           inception_v3[1][0]               
__________________________________________________________________________________________________
dense_2 (D

In [9]:
learning_rate = 0.0001
transfer_learning_epochs = 2000

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=learning_rate),
              metrics=['acc'])

checkpoint_dir = os.path.join(work_dir, 'checkpoint/ensemble')
log_dir = os.path.join(work_dir, 'log/ensemble')
if not os.path.exists(checkpoint_dir): os.mkdir(checkpoint_dir)

df = get_clinical_data(csv_file)

train_file_list = []
for path, dirs, files in os.walk(train_dir):
    train_file_list += [os.path.join(path, file) for file in files]    

validation_file_list = []
for path, dirs, files in os.walk(validation_dir):
    validation_file_list += [os.path.join(path, file) for file in files]    

model_path = os.path.join(checkpoint_dir, '{epoch:02d}-{val_acc:.4f}.hdf5')
checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_acc', period=10)
tensorboard = TensorBoard(log_dir=log_dir, write_graph=True, write_images=False)

history = model.fit_generator(
    custom_generator(train_file_list, df, is_train=True, non_image_data=True),
    steps_per_epoch=math.ceil(train_set_size/batch_size),
    callbacks=[checkpoint, tensorboard],
    validation_data=custom_generator(validation_file_list, df, is_train=False, non_image_data=True),
    epochs=transfer_learning_epochs,
    validation_steps=math.ceil(validation_set_size/batch_size))

Epoch 625/2000
17/17 [==============================] - 98s 6s/step - loss: 0.2596 - acc: 0.9014 - val_loss: 0.3180 - val_acc: 0.8712
Epoch 626/2000
17/17 [==============================] - 85s 5s/step - loss: 0.2673 - acc: 0.8968 - val_loss: 0.3012 - val_acc: 0.8761
Epoch 627/2000
17/17 [==============================] - 87s 5s/step - loss: 0.2538 - acc: 0.8964 - val_loss: 0.3134 - val_acc: 0.8674
Epoch 628/2000
17/17 [==============================] - 85s 5s/step - loss: 0.2541 - acc: 0.8991 - val_loss: 0.2981 - val_acc: 0.8751
Epoch 629/2000
17/17 [==============================] - 86s 5s/step - loss: 0.2608 - acc: 0.8950 - val_loss: 0.3145 - val_acc: 0.8722
Epoch 630/2000
17/17 [==============================] - 86s 5s/step - loss: 0.2581 - acc: 0.8993 - val_loss: 0.3089 - val_acc: 0.8780
Epoch 631/2000
17/17 [==============================] - 87s 5s/step - loss: 0.2573 - acc: 0.8991 - val_loss: 0.2999 - val_acc: 0.8771
Epoch 632/2000
17/17 [==============================] - 86s 5s

17/17 [==============================] - 87s 5s/step - loss: 0.2396 - acc: 0.9053 - val_loss: 0.3036 - val_acc: 0.8790
Epoch 748/2000
17/17 [==============================] - 84s 5s/step - loss: 0.2343 - acc: 0.9095 - val_loss: 0.3261 - val_acc: 0.8722
Epoch 749/2000
17/17 [==============================] - 85s 5s/step - loss: 0.2443 - acc: 0.9035 - val_loss: 0.3188 - val_acc: 0.8722
Epoch 750/2000
17/17 [==============================] - 86s 5s/step - loss: 0.2392 - acc: 0.9065 - val_loss: 0.3203 - val_acc: 0.8683
Epoch 751/2000
17/17 [==============================] - 86s 5s/step - loss: 0.2423 - acc: 0.9044 - val_loss: 0.3149 - val_acc: 0.8693
Epoch 752/2000
17/17 [==============================] - 84s 5s/step - loss: 0.2354 - acc: 0.9051 - val_loss: 0.3125 - val_acc: 0.8732
Epoch 753/2000
17/17 [==============================] - 84s 5s/step - loss: 0.2459 - acc: 0.9028 - val_loss: 0.3130 - val_acc: 0.8751
Epoch 754/2000
17/17 [==============================] - 84s 5s/step - loss: 0

17/17 [==============================] - 85s 5s/step - loss: 0.2244 - acc: 0.9123 - val_loss: 0.3169 - val_acc: 0.8712
Epoch 870/2000
17/17 [==============================] - 84s 5s/step - loss: 0.2325 - acc: 0.9069 - val_loss: 0.3292 - val_acc: 0.8654
Epoch 871/2000
17/17 [==============================] - 84s 5s/step - loss: 0.2297 - acc: 0.9104 - val_loss: 0.3305 - val_acc: 0.8654
Epoch 872/2000
17/17 [==============================] - 84s 5s/step - loss: 0.2246 - acc: 0.9083 - val_loss: 0.3231 - val_acc: 0.8693
Epoch 873/2000
17/17 [==============================] - 85s 5s/step - loss: 0.2315 - acc: 0.9053 - val_loss: 0.3022 - val_acc: 0.8771
Epoch 874/2000
17/17 [==============================] - 85s 5s/step - loss: 0.2263 - acc: 0.9104 - val_loss: 0.3076 - val_acc: 0.8761
Epoch 875/2000
17/17 [==============================] - 85s 5s/step - loss: 0.2354 - acc: 0.9104 - val_loss: 0.3010 - val_acc: 0.8771
Epoch 876/2000
17/17 [==============================] - 87s 5s/step - loss: 0

17/17 [==============================] - 87s 5s/step - loss: 0.2187 - acc: 0.9097 - val_loss: 0.3072 - val_acc: 0.8751
Epoch 992/2000
17/17 [==============================] - 85s 5s/step - loss: 0.2163 - acc: 0.9194 - val_loss: 0.2750 - val_acc: 0.8848
Epoch 993/2000
17/17 [==============================] - 85s 5s/step - loss: 0.2254 - acc: 0.9123 - val_loss: 0.3078 - val_acc: 0.8722
Epoch 994/2000
17/17 [==============================] - 85s 5s/step - loss: 0.2272 - acc: 0.9141 - val_loss: 0.3028 - val_acc: 0.8771
Epoch 995/2000
17/17 [==============================] - 86s 5s/step - loss: 0.2216 - acc: 0.9153 - val_loss: 0.3068 - val_acc: 0.8751
Epoch 996/2000
17/17 [==============================] - 86s 5s/step - loss: 0.2256 - acc: 0.9052 - val_loss: 0.2856 - val_acc: 0.8829
Epoch 997/2000
17/17 [==============================] - 85s 5s/step - loss: 0.2226 - acc: 0.9106 - val_loss: 0.3032 - val_acc: 0.8771
Epoch 998/2000
17/17 [==============================] - 86s 5s/step - loss: 0

Epoch 1052/2000
17/17 [==============================] - 85s 5s/step - loss: 0.2187 - acc: 0.9145 - val_loss: 0.3335 - val_acc: 0.8703
Epoch 1053/2000
17/17 [==============================] - 85s 5s/step - loss: 0.2219 - acc: 0.9116 - val_loss: 0.3223 - val_acc: 0.8674
Epoch 1054/2000
17/17 [==============================] - 86s 5s/step - loss: 0.2224 - acc: 0.9141 - val_loss: 0.3276 - val_acc: 0.8722
Epoch 1055/2000
17/17 [==============================] - 84s 5s/step - loss: 0.2135 - acc: 0.9180 - val_loss: 0.3139 - val_acc: 0.8712
Epoch 1056/2000
17/17 [==============================] - 85s 5s/step - loss: 0.2179 - acc: 0.9124 - val_loss: 0.3195 - val_acc: 0.8722
Epoch 1057/2000
17/17 [==============================] - 85s 5s/step - loss: 0.2138 - acc: 0.9201 - val_loss: 0.3238 - val_acc: 0.8664
Epoch 1058/2000
17/17 [==============================] - 83s 5s/step - loss: 0.2220 - acc: 0.9164 - val_loss: 0.3022 - val_acc: 0.8790
Epoch 1059/2000
17/17 [==============================] 

...

Epoch 1294/2000
17/17 [==============================] - 84s 5s/step - loss: 0.2190 - acc: 0.9116 - val_loss: 0.3491 - val_acc: 0.8674
Epoch 1295/2000
17/17 [==============================] - 86s 5s/step - loss: 0.2054 - acc: 0.9205 - val_loss: 0.2914 - val_acc: 0.8780
Epoch 1296/2000
17/17 [==============================] - 85s 5s/step - loss: 0.2073 - acc: 0.9251 - val_loss: 0.2755 - val_acc: 0.8887
Epoch 1297/2000
17/17 [==============================] - 85s 5s/step - loss: 0.2149 - acc: 0.9203 - val_loss: 0.2752 - val_acc: 0.8858
Epoch 1298/2000
17/17 [==============================] - 86s 5s/step - loss: 0.2113 - acc: 0.9162 - val_loss: 0.3265 - val_acc: 0.8751
Epoch 1299/2000
17/17 [==============================] - 84s 5s/step - loss: 0.2055 - acc: 0.9228 - val_loss: 0.2950 - val_acc: 0.8809
Epoch 1300/2000
17/17 [==============================] - 85s 5s/step - loss: 0.2143 - acc: 0.9143 - val_loss: 0.2770 - val_acc: 0.8858
Epoch 1301/2000
17/17 [==============================] 

Epoch 1415/2000
17/17 [==============================] - 86s 5s/step - loss: 0.2062 - acc: 0.9224 - val_loss: 0.3055 - val_acc: 0.8809
Epoch 1416/2000
17/17 [==============================] - 86s 5s/step - loss: 0.2041 - acc: 0.9217 - val_loss: 0.2989 - val_acc: 0.8790
Epoch 1417/2000
17/17 [==============================] - 85s 5s/step - loss: 0.2043 - acc: 0.9210 - val_loss: 0.2999 - val_acc: 0.8819
Epoch 1418/2000
17/17 [==============================] - 87s 5s/step - loss: 0.1996 - acc: 0.9219 - val_loss: 0.3100 - val_acc: 0.8780
Epoch 1419/2000
17/17 [==============================] - 84s 5s/step - loss: 0.2171 - acc: 0.9127 - val_loss: 0.2996 - val_acc: 0.8751
Epoch 1420/2000
17/17 [==============================] - 85s 5s/step - loss: 0.2174 - acc: 0.9141 - val_loss: 0.3025 - val_acc: 0.8771
Epoch 1421/2000
17/17 [==============================] - 86s 5s/step - loss: 0.2036 - acc: 0.9242 - val_loss: 0.2868 - val_acc: 0.8867
Epoch 1422/2000
17/17 [==============================] 

Epoch 1536/2000
17/17 [==============================] - 85s 5s/step - loss: 0.2027 - acc: 0.9242 - val_loss: 0.3176 - val_acc: 0.8761
Epoch 1537/2000
17/17 [==============================] - 85s 5s/step - loss: 0.2015 - acc: 0.9226 - val_loss: 0.3367 - val_acc: 0.8674
Epoch 1538/2000
17/17 [==============================] - 84s 5s/step - loss: 0.2057 - acc: 0.9208 - val_loss: 0.3041 - val_acc: 0.8819
Epoch 1539/2000
17/17 [==============================] - 87s 5s/step - loss: 0.2022 - acc: 0.9231 - val_loss: 0.3011 - val_acc: 0.8829
Epoch 1540/2000
17/17 [==============================] - 84s 5s/step - loss: 0.1977 - acc: 0.9228 - val_loss: 0.3059 - val_acc: 0.8800
Epoch 1541/2000
17/17 [==============================] - 85s 5s/step - loss: 0.2107 - acc: 0.9201 - val_loss: 0.3051 - val_acc: 0.8809
Epoch 1542/2000
17/17 [==============================] - 86s 5s/step - loss: 0.2010 - acc: 0.9244 - val_loss: 0.3399 - val_acc: 0.8722
Epoch 1543/2000
17/17 [==============================] 

Epoch 1657/2000
17/17 [==============================] - 84s 5s/step - loss: 0.1968 - acc: 0.9233 - val_loss: 0.3265 - val_acc: 0.8732
Epoch 1658/2000
17/17 [==============================] - 84s 5s/step - loss: 0.2005 - acc: 0.9231 - val_loss: 0.3025 - val_acc: 0.8867
Epoch 1659/2000
17/17 [==============================] - 86s 5s/step - loss: 0.2056 - acc: 0.9205 - val_loss: 0.3157 - val_acc: 0.8809
Epoch 1660/2000
 5/17 [=======>......................] - ETA: 1:05 - loss: 0.2290 - acc: 0.9187

KeyboardInterrupt: 

# Fine Tuning

In [24]:
layer_dict['dense_6'].trainable = False # shape=(2048, 1024)
layer_dict['dense_7'].trainable = False # shape=(3, 4)
layer_dict['dense_8'].trainable = False # shape=(4, 4)

for layer in conv_base.layers:
    layer.trainable = False
    
model_path = os.path.join(work_dir, 'checkpoint/ensemble/448-0.8858.hdf5')
model.load_weights(model_path)

layer_dict['dense_6'].trainable = True # shape=(2048, 1024)
layer_dict['dense_7'].trainable = True # shape=(3, 4)
layer_dict['dense_8'].trainable = True # shape=(4, 4)

for layer in conv_base.layers[:280]:
   layer.trainable = False
for layer in conv_base.layers[280:]:
   layer.trainable = True

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
inception_v3 (Model)            (None, 8, 8, 2048)   21802784    input_2[0][0]                    
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 3)            0                                            
__________________________________________________________________________________________________
global_average_pooling2d_2 (Glo (None, 2048)         0           inception_v3[1][0]               
__________________________________________________________________________________________________
dense_7 (D

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=learning_rate/10),
              metrics=['acc'])

checkpoint_dir = os.path.join(work_dir, 'checkpoint/ensemble_finetuning')
log_dir = os.path.join(work_dir, 'log/ensemble_finetuning')
if not os.path.exists(checkpoint_dir): os.mkdir(checkpoint_dir)

df = get_clinical_data(csv_file)

train_file_list = []
for path, dirs, files in os.walk(train_dir):
    train_file_list += [os.path.join(path, file) for file in files]    

validation_file_list = []
for path, dirs, files in os.walk(validation_dir):
    validation_file_list += [os.path.join(path, file) for file in files]    

model_path = os.path.join(checkpoint_dir, '{epoch:02d}-{val_acc:.4f}.hdf5')
checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_acc', period=10)
tensorboard = TensorBoard(log_dir=log_dir, write_graph=True, write_images=False)

history = model.fit_generator(
    custom_generator(train_file_list, df, is_train=True, non_image_data=True),
    steps_per_epoch=math.ceil(train_set_size/batch_size),
    callbacks=[checkpoint, tensorboard],
    validation_data=custom_generator(validation_file_list, df, is_train=False, non_image_data=True),
    epochs=transfer_learning_epochs,
    validation_steps=math.ceil(validation_set_size/batch_size))

Epoch 1/1000
17/17 [==============================] - 95s 6s/step - loss: 0.2476 - acc: 0.9051 - val_loss: 0.2905 - val_acc: 0.8829
Epoch 2/1000
17/17 [==============================] - 84s 5s/step - loss: 0.2484 - acc: 0.9035 - val_loss: 0.2870 - val_acc: 0.8809
Epoch 3/1000
17/17 [==============================] - 84s 5s/step - loss: 0.2441 - acc: 0.9028 - val_loss: 0.2871 - val_acc: 0.8819
Epoch 4/1000
17/17 [==============================] - 85s 5s/step - loss: 0.2424 - acc: 0.9062 - val_loss: 0.2881 - val_acc: 0.8838
Epoch 5/1000
17/17 [==============================] - 88s 5s/step - loss: 0.2519 - acc: 0.9005 - val_loss: 0.2863 - val_acc: 0.8858
Epoch 6/1000
17/17 [==============================] - 85s 5s/step - loss: 0.2479 - acc: 0.8984 - val_loss: 0.2864 - val_acc: 0.8848
Epoch 7/1000
17/17 [==============================] - 86s 5s/step - loss: 0.2396 - acc: 0.9056 - val_loss: 0.2860 - val_acc: 0.8829
Epoch 8/1000
17/17 [==============================] - 88s 5s/step - loss: 0.

17/17 [==============================] - 85s 5s/step - loss: 0.2244 - acc: 0.9101 - val_loss: 0.2836 - val_acc: 0.8790
Epoch 124/1000
17/17 [==============================] - 85s 5s/step - loss: 0.2282 - acc: 0.9134 - val_loss: 0.2810 - val_acc: 0.8790
Epoch 125/1000
17/17 [==============================] - 86s 5s/step - loss: 0.2246 - acc: 0.9161 - val_loss: 0.2818 - val_acc: 0.8800
Epoch 126/1000
17/17 [==============================] - 88s 5s/step - loss: 0.2193 - acc: 0.9157 - val_loss: 0.2814 - val_acc: 0.8800
Epoch 127/1000
17/17 [==============================] - 84s 5s/step - loss: 0.2151 - acc: 0.9157 - val_loss: 0.2831 - val_acc: 0.8780
Epoch 128/1000
17/17 [==============================] - 85s 5s/step - loss: 0.2286 - acc: 0.9120 - val_loss: 0.2853 - val_acc: 0.8771
Epoch 129/1000
17/17 [==============================] - 86s 5s/step - loss: 0.2290 - acc: 0.9154 - val_loss: 0.2795 - val_acc: 0.8761
Epoch 130/1000
17/17 [==============================] - 84s 5s/step - loss: 0

17/17 [==============================] - 86s 5s/step - loss: 0.2093 - acc: 0.9171 - val_loss: 0.2816 - val_acc: 0.8809
Epoch 246/1000
17/17 [==============================] - 87s 5s/step - loss: 0.2092 - acc: 0.9187 - val_loss: 0.2801 - val_acc: 0.8838
Epoch 247/1000
17/17 [==============================] - 86s 5s/step - loss: 0.2031 - acc: 0.9235 - val_loss: 0.2791 - val_acc: 0.8858
Epoch 248/1000
17/17 [==============================] - 86s 5s/step - loss: 0.2096 - acc: 0.9203 - val_loss: 0.2798 - val_acc: 0.8848
Epoch 249/1000
17/17 [==============================] - 86s 5s/step - loss: 0.2217 - acc: 0.9141 - val_loss: 0.2804 - val_acc: 0.8809
Epoch 250/1000
17/17 [==============================] - 85s 5s/step - loss: 0.2095 - acc: 0.9152 - val_loss: 0.2806 - val_acc: 0.8819
Epoch 251/1000
17/17 [==============================] - 86s 5s/step - loss: 0.2053 - acc: 0.9171 - val_loss: 0.2820 - val_acc: 0.8809
Epoch 252/1000
17/17 [==============================] - 86s 5s/step - loss: 0

17/17 [==============================] - 83s 5s/step - loss: 0.2008 - acc: 0.9233 - val_loss: 0.2848 - val_acc: 0.8867
Epoch 368/1000
17/17 [==============================] - 86s 5s/step - loss: 0.2050 - acc: 0.9230 - val_loss: 0.2853 - val_acc: 0.8877
Epoch 369/1000
17/17 [==============================] - 85s 5s/step - loss: 0.1957 - acc: 0.9221 - val_loss: 0.2858 - val_acc: 0.8858
Epoch 370/1000
17/17 [==============================] - 87s 5s/step - loss: 0.2066 - acc: 0.9196 - val_loss: 0.2878 - val_acc: 0.8819
Epoch 371/1000
17/17 [==============================] - 87s 5s/step - loss: 0.1954 - acc: 0.9254 - val_loss: 0.2859 - val_acc: 0.8819
Epoch 372/1000
17/17 [==============================] - 85s 5s/step - loss: 0.2015 - acc: 0.9251 - val_loss: 0.2832 - val_acc: 0.8829
Epoch 373/1000
17/17 [==============================] - 84s 5s/step - loss: 0.1941 - acc: 0.9253 - val_loss: 0.2830 - val_acc: 0.8848
Epoch 374/1000
17/17 [==============================] - 85s 5s/step - loss: 0

17/17 [==============================] - 85s 5s/step - loss: 0.1925 - acc: 0.9290 - val_loss: 0.2885 - val_acc: 0.8790
Epoch 490/1000
17/17 [==============================] - 85s 5s/step - loss: 0.1892 - acc: 0.9336 - val_loss: 0.2866 - val_acc: 0.8829
Epoch 491/1000
17/17 [==============================] - 86s 5s/step - loss: 0.1932 - acc: 0.9203 - val_loss: 0.2856 - val_acc: 0.8800
Epoch 492/1000
17/17 [==============================] - 86s 5s/step - loss: 0.1837 - acc: 0.9346 - val_loss: 0.2883 - val_acc: 0.8809
Epoch 493/1000
17/17 [==============================] - 86s 5s/step - loss: 0.1813 - acc: 0.9366 - val_loss: 0.2898 - val_acc: 0.8800
Epoch 494/1000
17/17 [==============================] - 86s 5s/step - loss: 0.1953 - acc: 0.9258 - val_loss: 0.2814 - val_acc: 0.8838
Epoch 495/1000
17/17 [==============================] - 86s 5s/step - loss: 0.1889 - acc: 0.9304 - val_loss: 0.2842 - val_acc: 0.8809
Epoch 496/1000
17/17 [==============================] - 84s 5s/step - loss: 0

17/17 [==============================] - 85s 5s/step - loss: 0.1825 - acc: 0.9293 - val_loss: 0.2832 - val_acc: 0.8838
Epoch 612/1000
17/17 [==============================] - 85s 5s/step - loss: 0.1729 - acc: 0.9311 - val_loss: 0.2877 - val_acc: 0.8838
Epoch 613/1000
17/17 [==============================] - 86s 5s/step - loss: 0.1840 - acc: 0.9332 - val_loss: 0.2831 - val_acc: 0.8887
Epoch 614/1000
17/17 [==============================] - 85s 5s/step - loss: 0.1868 - acc: 0.9279 - val_loss: 0.2800 - val_acc: 0.8848
Epoch 615/1000
17/17 [==============================] - 86s 5s/step - loss: 0.1775 - acc: 0.9334 - val_loss: 0.2766 - val_acc: 0.8877
Epoch 616/1000
17/17 [==============================] - 84s 5s/step - loss: 0.1799 - acc: 0.9300 - val_loss: 0.2732 - val_acc: 0.8867
Epoch 617/1000
17/17 [==============================] - 85s 5s/step - loss: 0.1815 - acc: 0.9304 - val_loss: 0.2768 - val_acc: 0.8877
Epoch 618/1000
17/17 [==============================] - 85s 5s/step - loss: 0

In [18]:
from sklearn.metrics import auc
df = get_clinical_data(csv_file)

test_file_list, img_test, clinical_test, y_test = get_image_data(test_dir, df)
#test_file_list = []
#for path, dirs, files in os.walk(test_dir):
#    test_file_list += [os.path.join(path, file) for file in files]
#count = len(test_file_list)

img_test = preprocess_input(img_test, is_train=False)
y_pred = model.predict([img_test, clinical_test])

scores = model.evaluate_generator(custom_generator(test_file_list, df, is_train=False, non_image_data=True), steps=math.ceil(test_set_size/batch_size))
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

fpr, tpr, thresholds = roc_curve(y_test, y_pred)
auc = auc(fpr, tpr)
print('auc=%0.3f' % auc)

total images = 885
acc: 83.73%
auc=0.924


In [ ]:
for file, clinical, y, pred in zip(file_list, clinical_test, y_test, y_pred):
    print(file, y, pred[0])